In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import pandas as pd

from tqdm import tqdm


2024-05-20 18:17:45.237809: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 18:17:46.561932: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 18:17:46.574280: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 18:17:51.834985: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1.Create model

In [2]:
# Define the CNN model
def create_cnn_model(input_shape):
    model = models.Sequential()

    # Add Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    # Add Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))  # 2 output neurons for linear and angular speed

    return model

# Create an instance of the model
input_shape = (200, 200, 1)  # Define the input shape based on your camera image dimensions
model = create_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Using Mean Squared Error as loss for regression

# Print model summary
model.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 135424)            0

# 2. Load data

In [14]:
import os
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import img_to_array

def load_img(path):
    try:
        path = os.path.join('/home/miquel/Escritorio/VAR/P2',str(path))
        image = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (input_shape[0], input_shape[1]))
        image = image / 255.0
        image = img_to_array(image)
        image = image.reshape((input_shape[0], input_shape[1]))
        return image
    except:
        print(path)
        return None


paths = [f'data/dataset{i+1}.json' for i in range(4)]
dfs = []
for path in paths:
    dfs.append(pd.read_json(path))

df = pd.concat(dfs, axis=0)
df = df.reset_index()
images = []
labels = []
for i, path in tqdm(enumerate(df['path'])):
    images.append(load_img(path))
    labels.append(np.array([np.float32(df.at[i,'V']), np.float32(df.at[i,'W'])]))

# tqdm.pandas()
# images = np.array(df['path'].progress_apply(lambda x: load_img(x)).values)
# labels = np.array(df.progress_apply(
#     lambda x: 
#         np.array([np.float32(x['V']), np.float32(x['W'])]),
#     axis=1).values)

#train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

images = np.array(images)
labels = np.array(labels)
images, labels = shuffle(images, labels, random_state=42)

idx = int(len(images)*0.75)
train_images, train_labels = images[:idx], labels[:idx]
val_images, val_labels = images[idx:], labels[idx:]

print(f"train_images shape: {train_images.shape}, dtype: {train_images.dtype}")
print(f"train_labels shape: {train_labels.shape}, dtype: {train_labels.dtype}")
print(f"val_images shape: {val_images.shape}, dtype: {val_images.dtype}")
print(f"val_labels shape: {val_labels.shape}, dtype: {val_labels.dtype}")


3204it [00:17, 181.62it/s]


train_images shape: (2403, 200, 200), dtype: float32
train_labels shape: (2403, 2), dtype: float32
val_images shape: (801, 200, 200), dtype: float32
val_labels shape: (801, 2), dtype: float32


In [17]:
print(images.shape)
print(images[0].shape)
print(images[0][0].shape)
print(images[0][0][0].shape)
print(images[0][0][0])



for i, img in enumerate(images):
    print(img.shape, end='')
    if(i%13==0):
        print()


(2202,)
(200, 200)
(200,)
()
0.47843137
(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)(200, 200)
(200, 200)(200, 200)(200, 200)(2

# 3. Train

In [15]:

model = create_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Define callbacks (optional but useful for saving the best model)
checkpoint_path = "best_model.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model
batch_size = 32
epochs = 10

history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[checkpoint])

# Evaluate the model on the validation set
loss, mae = model.evaluate(val_images, val_labels)
print("Validation Loss:", loss)
print("Validation Mean Absolute Error:", mae)

# Load the best model
best_model = tf.keras.models.load_model(checkpoint_path)

# Make predictions on the validation set using the best model
predictions = best_model.predict(val_images)

# Optionally, you can save the best model for later use
best_model.save("data/models/best_model_trained.h5")

2024-05-20 23:09:15.789719: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 384480000 exceeds 10% of free system memory.


Epoch 1/10
76/76 [==============================] - ETA: 0s - loss: 1.3136 - mae: 0.6739
Epoch 1: val_loss improved from inf to 0.12856, saving model to best_model.h5


/home/miquel/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


76/76 [==============================] - 234s 3s/step - loss: 1.3136 - mae: 0.6739 - val_loss: 0.1286 - val_mae: 0.2081
Epoch 2/10
76/76 [==============================] - ETA: 0s - loss: 0.1163 - mae: 0.2048
Epoch 2: val_loss improved from 0.12856 to 0.06241, saving model to best_model.h5
76/76 [==============================] - 236s 3s/step - loss: 0.1163 - mae: 0.2048 - val_loss: 0.0624 - val_mae: 0.1509
Epoch 3/10
76/76 [==============================] - ETA: 0s - loss: 0.0704 - mae: 0.1615
Epoch 3: val_loss improved from 0.06241 to 0.04530, saving model to best_model.h5
76/76 [==============================] - 233s 3s/step - loss: 0.0704 - mae: 0.1615 - val_loss: 0.0453 - val_mae: 0.1306
Epoch 4/10
76/76 [==============================] - ETA: 0s - loss: 0.0471 - mae: 0.1245
Epoch 4: val_loss did not improve from 0.04530
76/76 [==============================] - 234s 3s/step - loss: 0.0471 - mae: 0.1245 - val_loss: 0.0456 - val_mae: 0.1290
Epoch 5/10
76/76 [========================